In [2]:
import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np

# Load the uploaded CSV files
log_episode_path = 'D:\\Aprendizado_por_Reforco\\TradeM1\\4.17\\log_episode_85.csv'
bar_m15_path = 'D:\\dados\\bar_M15_V02_data_01-01-2023_a_31-08-2024.csv'
df = pd.read_csv(log_episode_path)

log_episode_df = pd.read_csv(log_episode_path)
bar_m15_df = pd.read_csv(bar_m15_path)

# Convert DateTime to pandas datetime format in bar_m15_df
bar_m15_df['DateTime'] = pd.to_datetime(bar_m15_df['DateTime'])

# Map entry and exit DateTime to log_episode_df based on entry_step and exit_step
log_episode_df['entry_datetime'] = log_episode_df['entry_step'].apply(lambda x: bar_m15_df.iloc[x]['DateTime'] if x < len(bar_m15_df) else pd.NaT)
log_episode_df['exit_datetime'] = log_episode_df['exit_step'].apply(lambda x: bar_m15_df.iloc[x]['DateTime'] if x < len(bar_m15_df) else pd.NaT)

# Use entry_datetime as the DateTime for plotting
log_episode_df['DateTime'] = log_episode_df['entry_datetime']

# Plot the profit progression over time
fig_profit = go.Figure()
fig_profit.add_trace(go.Scatter(
    x=log_episode_df['DateTime'],
    y=log_episode_df['profit'],
    mode='lines',
    name='Profit Progression',
    line=dict(color='blue')
))

# Set chart title and labels for profit progression
fig_profit.update_layout(
    title='Profit Progression Over Time',
    xaxis_title='Date',
    yaxis_title='Profit',
    template='plotly_white'
)

# Save the profit progression plot to a folder
output_folder = '4.5'
os.makedirs(output_folder, exist_ok=True)
fig_profit.write_html(os.path.join(output_folder, 'profit_progression.html'))

# Show the profit progression plot
fig_profit.show()

# Plot the cumulative profit over time
log_episode_df['cumulative_profit'] = log_episode_df['profit'].cumsum()

fig_cumulative_profit = go.Figure()
fig_cumulative_profit.add_trace(go.Scatter(
    x=log_episode_df['DateTime'],
    y=log_episode_df['cumulative_profit'],
    mode='lines',
    name='Cumulative Profit',
    line=dict(color='green')
))

# Set chart title and labels for cumulative profit
fig_cumulative_profit.update_layout(
    title='Cumulative Profit Over Time',
    xaxis_title='Date',
    yaxis_title='Cumulative Profit',
    template='plotly_white'
)

# Save the cumulative profit plot to a folder
fig_cumulative_profit.write_html(os.path.join(output_folder, 'cumulative_profit.html'))

# Show the cumulative profit plot
fig_cumulative_profit.show()

# Calculate metrics: Drawdown, Sharpe Ratio, Sortino Ratio
def calculate_drawdown(profit_series):
    cumulative = profit_series.cumsum()
    high_watermark = cumulative.cummax()
    drawdown = cumulative - high_watermark
    return drawdown

drawdown = calculate_drawdown(log_episode_df['profit'])
max_drawdown = drawdown.min()

# Calculate Sharpe Ratio
average_profit = log_episode_df['profit'].mean()
profit_std = log_episode_df['profit'].std()
sharpe_ratio = average_profit / profit_std if profit_std != 0 else 0

# Calculate Sortino Ratio
negative_profit = log_episode_df[log_episode_df['profit'] < 0]['profit']
negative_std = negative_profit.std()
sortino_ratio = average_profit / negative_std if negative_std != 0 else 0

# Calculate Winrate
num_wins = len(log_episode_df[log_episode_df['profit'] > 0])
num_losses = len(log_episode_df[log_episode_df['profit'] <= 0])
num_trades = len(log_episode_df)
winrate = (num_wins / num_trades) * 100 if num_trades > 0 else 0
# soma a coluna profit
profittotal = df['profit'].sum()

# Print metrics
print(f"Max Drawdown: {max_drawdown}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"Sortino Ratio: {sortino_ratio}")
print(f"Winrate: {winrate:.2f}%")
print(f"Total Trades: {num_trades}")
print(f"Winning Trades: {num_wins}")
print(f"Losing Trades: {num_losses}")
print(f"Profit Total: {profittotal}")


Max Drawdown: -914.0
Sharpe Ratio: 0.07334804669502423
Sortino Ratio: 0.07801312633711767
Winrate: 60.24%
Total Trades: 1436
Winning Trades: 865
Losing Trades: 571
Profit Total: 7195.25


In [24]:
df.head(10)
# salva o dataframe em txt
#df.to_csv('4.7\\df.txt', sep='\t', index=False)



,type,entry_step,entry_price,entry_datetime,exit_step,exit_price,exit_datetime,profit
0,buy,22,12472.00,2023-01-05 10:45:00,43,12298.75,2023-01-05 16:00:00,-173.50
1,buy,44,12320.00,2023-01-05 16:15:00,52,12279.25,2023-01-05 18:15:00,-41.00
2,buy,110,12272.25,2023-01-06 10:45:00,122,12331.75,2023-01-06 13:45:00,59.25
3,buy,124,12363.25,2023-01-06 14:15:00,126,12243.75,2023-01-06 14:45:00,-119.75
4,buy,127,12271.50,2023-01-06 15:00:00,129,12348.75,2023-01-06 15:30:00,77.00
5,buy,130,12398.00,2023-01-06 15:45:00,131,12443.25,2023-01-06 16:00:00,45.00
6,buy,132,12439.25,2023-01-06 16:15:00,140,12488.00,2023-01-06 18:15:00,48.50
7,sell,190,12613.25,2023-01-09 10:45:00,193,12605.75,2023-01-09 11:30:00,7.25
8,sell,194,12609.00,2023-01-09 11:45:00,196,12619.75,2023-01-09 12:15:00,-11.00
9,sell,197,12620.00,2023-01-09 12:30:00,201,12641.00,2023-01-09 13:30:00,-21.25
